In [2]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [3]:
sys.setrecursionlimit(5000)

In [ ]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung,
)

from compute_ET_green import compute_et_green, calculate_band_std_dev

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
)
from utils import date_utils, ee_utils

from typing import List, Tuple

import ee
import geemap

In [5]:
ee.Initialize(project="thurgau-irrigation")

## Compute and export ET Green


In [6]:
TIME_STEPS = 12
TIME_STEP_TYPE = "monthly"
YEARS = range(2018, 2023)

In [7]:
# DOUBLE_CROPPING_IMAGE = ee.Image(
#     "projects/thurgau-irrigation/assets/Zuerich/crop_vegetation_period_zh_2022"
# )

In [8]:
# path_to_WaPOR_ET_asset = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal_test_refactor"


# ET_COLLECTIONs = ee_utils.merge_collections(
#     years=YEARS, asset_name=path_to_WaPOR_ET_asset
# )

# ET_COLLECTION = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Zuerich/ET_WaPOR_10m_dekadal_2022"
# ).map(lambda img: back_to_float(img, 100))

ET_COLLECTION = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/Landsat_ET_gap_filled_2018-2022"
).map(lambda img: back_to_float(img, 100))

ee_utils.print_value_ranges(ET_COLLECTION, "fitted_ET")

Image 1: Min = -20.19, Max = 30.75
Image 2: Min = -12.52, Max = 35.92
Image 3: Min = -8.88, Max = 56.41
Image 4: Min = -14.70, Max = 87.69
Image 5: Min = -17.92, Max = 119.38
Image 6: Min = -17.54, Max = 138.54
Image 7: Min = -13.65, Max = 141.17
Image 8: Min = -8.89, Max = 127.93
Image 9: Min = -7.76, Max = 102.25
Image 10: Min = -5.09, Max = 68.56
Image 11: Min = -9.93, Max = 43.62
Image 12: Min = -18.18, Max = 29.27
Image 13: Min = -17.11, Max = 24.05
Image 14: Min = -8.64, Max = 29.30
Image 15: Min = -10.06, Max = 53.06
Image 16: Min = -15.76, Max = 86.02
Image 17: Min = -19.01, Max = 116.93
Image 18: Min = -18.66, Max = 136.21
Image 19: Min = -14.79, Max = 138.66
Image 20: Min = -8.57, Max = 125.07
Image 21: Min = -6.14, Max = 100.65
Image 22: Min = -6.44, Max = 66.92
Image 23: Min = -8.47, Max = 37.06
Image 24: Min = -14.05, Max = 22.65
Image 25: Min = -16.43, Max = 17.35
Image 26: Min = -6.41, Max = 24.92
Image 27: Min = -11.90, Max = 54.88
Image 28: Min = -16.97, Max = 87.81
Im

In [9]:
ET_COLLECTION.first().bandNames().getInfo()

['fitted_ET']

In [10]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [ ]:
def process_and_export_et_green(
    et_collection_list: ee.List,
    year: int,
    aoi: ee.Geometry,
    time_steps: int = 36,
    time_step_type: str = "dekadal",
    double_cropping_image: ee.Image = None,
) -> List[ee.batch.Task]:
    """
    Process and export ET green images for a given year.

    Args:
        et_collection (ee.List): List of ET images
        year (int): Year to process
        aoi (ee.Geometry): Area of interest
        time_steps (int): Number of time steps (36 for dekadal, 12 for monthly)
        time_step_type (str): Type of time step ("dekadal" or "monthly")
        double_cropping_image (ee.Image): Double cropping image

    Returns:
        List[ee.batch.Task]: List of export tasks
    """
    nutzung_collection = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{year}_area"
    )

    # nutzung_collection = nutzung_collection.map(
    #     lambda feature: feature.set("nutzung", feature.get("NUTZUNG"))
    # )

    jurisdictions = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Thurgau/thurgau_jurisdictions_{year}_wgs84"
    )

    double_cropping = ee.Image(double_cropping_image)

    # Prepare crop filters
    not_irrigated_crops = get_crops_to_exclude()
    rainfed_crops = get_rainfed_reference_crops()
    exclude_filter, rainfed_filter = create_crop_filters(
        not_irrigated_crops, rainfed_crops
    )

    # Process nutzung collection
    nutzung_with_double_crop = add_double_cropping_info(
        nutzung_collection, double_cropping
    )
    _, rainfed_fields = filter_crops(
        nutzung_with_double_crop, exclude_filter, rainfed_filter
    )

    tasks = []
    for i in range(time_steps):
        # Get time step info
        if time_step_type == "dekadal":
            dekadal = i % 3 + 1
            month = i // 3 + 1
            time_step_name = f"{month:02d}_D{dekadal}"
        elif time_step_type == "monthly":
            month = i + 1
            time_step_name = f"{month:02d}"
        else:
            raise ValueError("time_step_type must be either 'dekadal' or 'monthly'")

        # Get ET image and compute ET green
        et_image = ee.Image(et_collection_list.get(i))
        et_green = compute_et_green(
            et_image, rainfed_fields, jurisdictions, et_band_name="fitted_ET"
        )

        et_green = back_to_int(et_green, 100)

        # Export result
        task_name = (
            f"ET_green_landsat_jurisdiction_TG_{time_step_type}_{year}-{time_step_name}"
        )
        asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_green_landsat_30m_jurisdiction_TG_{time_step_type}_2018-2022/{task_name}"

        task = export_image_to_asset(et_green, asset_id, task_name, year, aoi, scale=30)
        tasks.append(task)

    return tasks

In [ ]:
# for year in YEARS:

#     et_collection_list = ET_COLLECTION.filterDate(
#         f"{year}-01-01", f"{year}-12-31"
#     ).toList(TIME_STEPS)

#     DOUBLE_CROPPING_IMAGE = ee.Image(f"projects/thurgau-irrigation/assets/Thurgau/VegetationPeriod/crop_veg_period_{year}")


#     tasks = process_and_export_et_green(
#         et_collection_list, year, AOI, TIME_STEPS, TIME_STEP_TYPE, DOUBLE_CROPPING_IMAGE
#     )

#     print(f"Started {len(tasks)} export tasks.")

In [13]:
# collection.size().getInfo()

# date_utils.print_collection_dates(collection)

### Sanity check


In [ ]:
check_collection = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/ET_green_landsat_30m_jurisdiction_TG_monthly_2018-2022"
).map(lambda img: back_to_float(img, 100))

check_collection.first().bandNames().getInfo()

['ET_green']

In [15]:
date_utils.print_collection_dates(check_collection)
print_value_ranges(check_collection, "ET_green")

Dates of images in the collection:
2018-01-01
2018-02-01
2018-03-01
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01
2018-11-01
2018-12-01
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2019-11-01
2019-12-01
2020-01-01
2020-02-01
2020-03-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2020-11-01
2020-12-01
2021-01-01
2021-02-01
2021-03-01
2021-04-01
2021-05-01
2021-06-01
2021-07-01
2021-08-01
2021-09-01
2021-10-01
2021-11-01
2021-12-01
2022-01-01
2022-02-01
2022-03-01
2022-04-01
Image 1: Min = -3.32, Max = 5.68
Image 2: Min = 5.89, Max = 16.87
Image 3: Min = 24.36, Max = 39.67
Image 4: Min = 47.38, Max = 69.71
Image 5: Min = 67.50, Max = 97.84
Image 6: Min = 79.50, Max = 114.97
Image 7: Min = 81.44, Max = 117.45
Image 8: Min = 72.26, Max = 105.88
Image 9: Min = 55.16, Max = 82.42
Image 10: Min = 33.59, Max = 47.80
Image 11: Min = 12.88, Max = 24.60
Image 12: Min = 

In [ ]:
Map = geemap.Map()

image = ee.Image(check_collection.toList(check_collection.size()).get(-1))

vis_params = {
    "bands": ["ET_green"],
    "min": 0,
    "max": 100,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}

Map.center_object(AOI, 12)
Map.addLayer(image, vis_params, "ET_green")

Map

Map(center=[47.56858787382066, 9.092720596553875], controls=(WidgetControl(options=['position', 'transparent_b…